In [29]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("trees").getOrCreate()

In [2]:
data=spark.read.csv("/content/College.csv",inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [6]:
assembler=VectorAssembler(inputCols= ['Apps',
                                      'Accept',
                                      'Enroll',
                                      'Top10perc',
                                      'Top25perc',
                                      'F_Undergrad',
                                      'P_Undergrad',
                                      'Outstate',
                                      'Room_Board',
                                      'Books',
                                      'Personal',
                                      'PhD',
                                      'Terminal',
                                      'S_F_Ratio',
                                      'perc_alumni',
                                      'Expend',
                                      'Grad_Rate'],outputCol="features")

In [7]:
output=assembler.transform(data)

In [8]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer=StringIndexer(inputCol="Private",outputCol="Privateindex")

In [11]:
output_fixed=indexer.fit(output).transform(output)

In [13]:
output_fixed.show()

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|            features|Privateindex|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|         0.0|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30

In [14]:
final_data=output_fixed.select(["features","Privateindex"])

In [15]:
final_data.show()

+--------------------+------------+
|            features|Privateindex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [16]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [32]:
dtc=DecisionTreeClassifier(labelCol="Privateindex",featuresCol="features")
rfc=RandomForestClassifier(labelCol="Privateindex",featuresCol="features",numTrees=200)
gbt=GBTClassifier(labelCol="Privateindex",featuresCol="features")

In [33]:
dtc_model=dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model=gbt.fit(train_data)

In [34]:
dtc_preds=dtc_model.transform(test_data)
rfc_preds=rfc_model.transform(test_data)
gbt_preds=gbt_model.transform(test_data)

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [36]:
evaluator=BinaryClassificationEvaluator(labelCol="Privateindex")

In [37]:
print("DTC ACC: ",evaluator.evaluate(dtc_preds))
print("RFC ACC: ",evaluator.evaluate(rfc_preds))
print("GBT ACC: ",evaluator.evaluate(gbt_preds))

DTC ACC:  0.9133040935672514
RFC ACC:  0.97007797270955
GBT ACC:  0.9310428849902533


In [38]:
evaluator_gbt=BinaryClassificationEvaluator(labelCol="Privateindex",rawPredictionCol="prediction" )

In [39]:
print("GBT ACC: ",evaluator_gbt.evaluate(gbt_preds))

GBT ACC:  0.877046783625731
